In [1]:
import pandas3js as pjs
print(pjs.__version__)
import jsonextended as ejson
print(ejson.__version__)
import pandas as pd

0.0.3
0.1.3.3


In [17]:
data = ejson.json_to_dict(ejson.get_test_path())
ejson.dict_pprint(data, depth=3)

dir1: 
  dir1_1: 
    file1_1: {...}
  file1: 
    initial: {...}
    meta: {...}
    optimised: {...}
    units: {...}
  file2: 
    initial: {...}
    meta: {...}
    optimisation: {...}
    optimised: {...}
    units: {...}
dir2: 
  file1: 
    initial: {...}
    meta: {...}
    optimisation: {...}
    optimised: {...}
    units: {...}
dir3: 


In [18]:
data = ejson.dict_remove_paths(data, ['units'])
energies = ejson.dict_filter_keys(data, ['energy'])
pd.Series(ejson.dict_flatten(energies)).describe()

count       89.000000
mean    -24062.286741
std          0.012361
min     -24062.293964
25%     -24062.293727
50%     -24062.293491
75%     -24062.278072
max     -24062.207939
dtype: float64

In [21]:
energies = ejson.units.apply_unitschema(energies, {'energy':'eV'})
energies = ejson.units.apply_unitschema(energies, {'energy':'kcal'},as_quantity=False)
pd.Series(ejson.dict_flatten(energies)).describe()

count    8.900000e+01
mean    -9.214157e-19
std      4.733481e-25
min     -9.214159e-19
25%     -9.214159e-19
50%     -9.214159e-19
75%     -9.214153e-19
max     -9.214126e-19
dtype: float64

In [23]:
optimisation = ejson.dict_multiindex(data,['dir1','file2','optimisation','steps'])
optsteps = sorted(optimisation.keys(), key=ejson.core._natural_keys)
print(optsteps)

[u'2', u'3', u'4', u'5', u'6', u'7', u'8', u'9', u'10', u'11', u'12', u'13', u'14', u'15', u'16', u'17', u'18', u'19', u'20', u'21', u'22', u'23', u'24', u'25', u'26', u'27', u'28', u'29', u'30', u'31', u'32', u'33', u'34', u'35', u'36', u'37', u'38', u'39', u'40', u'41', u'42', u'43', u'44', u'45', u'46', u'47', u'48', u'49', u'50', u'51', u'52', u'53', u'54', u'55', u'56', u'57', u'58', u'59', u'60', u'61', u'62', u'63', u'64', u'65', u'66', u'67', u'68', u'69', u'70', u'71', u'72', u'73', u'74', u'75', u'76', u'77', u'78', u'79', u'80', u'81', u'82', u'83', u'84', u'85', u'86', u'87', u'88', u'89', u'90']


In [32]:
ejson.dict_pprint(optimisation['2'],depth=None,no_values=False)

crystallographic: 
  geometry: 
    assym:         [True, False, False, False, True, False, False, False, True, 
                   False, False, False, True, False, False, False, True, False, 
                   False, False, True, False, False, False, True, False, False, 
                   False, True, False]
    atomic_number: [16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 
                   16, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26]
    id:            [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 
                   18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
    label:         ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 
                   'S', 'S', 'S', 'S', 'FE', 'FE', 'FE', 'FE', 'FE1', 'FE1', 
                   'FE1', 'FE1', 'FE', 'FE', 'FE', 'FE', 'FE1', 'FE1']
    x/a:           [-0.09616823805925, 0.09616823805925, 0.09616823805925, 
                   -0.09616823805925, -0.1508611685218, -0.3491388314

In [29]:
ejson.dict_to_html(optimisation['2'])

In [30]:
geometry = pjs.GeometricCollection()


In [38]:
import ipywidgets as widgets

collection = pjs.GeometricCollection()

scene = pjs.create_js_scene_view(collection,add_objects=True,add_labels=True)

camera, renderer = pjs.create_jsrenderer(scene,view=(1,-1,-1,1))

select = widgets.SelectionSlider(
    options=optsteps,
#    value=sortkeys[0],
    description='Configuration:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
#     readout_format='i',
     slider_color='black'
)
def format_label(s):
    s = ''.join([i for i in s if not i.isdigit()])
    return s.lower().capitalize()

def change_config(step,label=False):
    sdict = optimisation[step]['crystallographic']['geometry']
    df = pd.DataFrame(sdict)
    df.rename(columns={'x/a':'x','y/b':'y','z/c':'z'},
             inplace=True)
    #df['otype'] = 'Sphere'
    df.label = df.label.apply(format_label)
    df['radius'] = 0.1
    df['color'] = 'red'
    df.set_value(df['label']=='Fe','color','blue')
    df['label_color'] = df.color
    df['visible'] = not label
    df['label_visible'] = label
    collection.change_by_df(df,columns=['id','label','x','y','z','radius',
                                       'color', 'label_color','visible','label_visible'],
                            otype_default='pandas3js.Sphere')
toggle=widgets.Checkbox(
    value=False,
    description='View Label:',
    disabled=False
)
def handle_toggle(change):
    change_config(select.value, change.new)
toggle.observe(handle_toggle,names='value')

def handle_slider(change):
    change_config(change.new, toggle.value)
select.observe(handle_slider,names='value')
select.value = optsteps[-1]

widgets.VBox([widgets.HBox([select,toggle]),renderer])